In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
import os
import contextlib
import models

%run -i experiments.py

In [ ]:
cwd = os.getcwd()
base = './results'
res_path = os.path.join(cwd,base)
res_path


In [ ]:
def walk(path, extension = ".pyT"):
    for root, dirs, files in os.walk(path):
        files = list(filter(lambda file: file.endswith(extension), files))
        if len(files) == 0:
            continue
        yield (root, dirs, files)

def extract_metadata(root):
    
    index = root.index('epochs')
    root = root[index:].split('/')
    
    dictionary = dict()  
    dictionary.update([item.split('=') for item in root[0].split('-')])
    
    if len(root) > 1:
        dictionary['is_tied'] = False
        dictionary['split_epoch'] = int(root[-1].split('_')[-1])
    
    else:
        dictionary['is_tied'] = True
        dictionary['split_epoch'] = 0
    
    return dictionary

@contextlib.contextmanager
def remember_cwd():
    curdir = os.getcwd()
    try:
        yield
    finally:
        os.chdir(curdir)
        
def preprocess(data):
    # remove the initial result
    for key in data.keys():
        if key not in ['train_loss', 'test_loss', 'train_acc', 'test_acc', 'checkpoints']:
            data[key].insert(0, data[key][0])
    
    return pd.DataFrame(data)


In [ ]:
evaluation_history = []
roots = []

## Builds the evaluation data frame
with remember_cwd():
    for root, dirs, files in walk(res_path, extension = ".hist"):
        os.chdir(root)
        metadata = extract_metadata(root)
        for file in files:
            if 'evaluation' in file:
                package = preprocess(torch.load(file, map_location=device))
                for key, val in metadata.items():
                    package[key] = val
                evaluation_history.append( package )
                roots.append(root)

evaluation_history_df = pd.concat( evaluation_history, axis = 0) \
                          .assign(adjusted_epoch = lambda x: x.checkpoints + x.split_epoch,
                                  experiment_type = lambda x: 'CNN lr=' + x.cnn_lr.astype(str) + ', LCN lr=' + x.lc_lr.astype(str))

evaluation_history_per_experiment = evaluation_history_df \
    .groupby(['experiment_type', 'split_epoch']) \
    .apply(lambda x: x.sort_values(['test_acc'], ascending=False).iloc[0,:])

best_elcn = evaluation_history_per_experiment.sort_values(['test_acc'], ascending=False).iloc[0,].split_epoch



The following produces a table to compare the best eLCN and CNN under different learning rates.


In [ ]:
evaluation_history_per_experiment.loc[(slice(None), [0, best_elcn]), :]


